<a href="https://colab.research.google.com/github/maggawron/S_P500/blob/main/S%26P_500_Analytics_data_loading.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**S&P 500 Analytics - data loading** *by Magdalena Gawron*
Welcome to the first part of my S&P 500 analytics project. This part will focus on fetching stock tickers and names, historical price data and analyst recommendations via webscraping and Yahoo APIs 

All the data structures generated in this notebook have been saved on my github repository in the [Data](https://github.com/maggawron/S_P500/tree/main/Data) folder

#Imports 
Import all the key packages needed to fetch the data

In [8]:
#@title Imports
import pandas as pd
import numpy as np
import sys
from datetime import date
from copy import deepcopy

!pip install yfinance
!pip install yahoofinancials
!pip install Yahoo-ticker-downloader

import bs4 as bs
import pickle
import requests
import yfinance as yf
from yahoofinancials import YahooFinancials

import pickle

from datetime import timedelta

from google.colab import output
output.clear()

print("done")

done


#Get all basic data about S&P 500 stocks
Scrape stocks' tickers and stocks full names from Wikipedia page.
This process can be skipped via using the next Colab file with analytics

In [5]:
#@title Scrape all SP500 tickers from Wikipedia page
def sp500_tickers():
  """scrape all S&P 500 tickers from Wikipedia and return them as a list"""
  resp = requests.get('http://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
  soup = bs.BeautifulSoup(resp.text, 'lxml')
  table = soup.find('table', {'class': 'wikitable sortable'})
  tickers = []
  all_names = []
  for row in table.findAll('tr')[1:]:
    ticker = row.findAll('td')[0].text
    name = row.findAll('td')[1].text
    all_names.append(name.strip())
    tickers.append(ticker.strip())
        
  with open("sp500tickers.pickle","wb") as f:
    pickle.dump(tickers,f)     
  return tickers, all_names

assets, stock_names = sp500_tickers()

#Remove couple of tickers triggering YFinance API errors   
to_pop = ["BRK.B","BF.B", "CI", "STE", "ZBH"]
for i in to_pop:
  ind = assets.index(i)
  assets.pop(ind)
  stock_names.pop(ind)

In [6]:
#Save the data to a file
with open('assets_tickers.pkl', 'wb') as f:
  pickle.dump(assets, f)
with open('assets_names.pkl', 'wb') as f:
  pickle.dump(stock_names, f)

#Download the US stock prices data
This process takes quite some time to complete. It can be skipped via using the next Colab file with analytics

In [ ]:
#@title Download stock price data for S&P 500 until today
date_start = '2013-01-01' #@param 
date_end = str(date.today() - timedelta(days=1))
interval = 'daily' #@param 

#Generate a dataframe with all stock prices 
all_data = pd.DataFrame({"date":[str(date.today())]})

for ticker in assets:
  yahoo_financials = YahooFinancials(ticker)
  data = yahoo_financials.get_historical_price_data(start_date=date_start, 
                                                    end_date=date_end, 
                                                    time_interval=interval)
  
  df = pd.DataFrame(data[ticker]['prices'])
  prices_df = pd.DataFrame({"date": df["formatted_date"], ticker: df['adjclose']}) 
  all_data = pd.merge(all_data, prices_df, how="outer", on="date")
  
#All_data is the dataframe containing all S&P 500 stock prices available on Yahoo platform

In [10]:
#@title Pick 80% of the stocks with the longest presence on the stock market
#Exclude all stocks with short track record
mean_no_records = all_data.nunique().quantile(0.2)

for ticker in assets:
  if ticker in all_data:
    if all_data[ticker].nunique() < mean_no_records:
      all_data.drop(columns = ticker, inplace=True)

all_data.dropna(inplace=True)
all_data.shape

(1963, 401)

In [11]:
#Save the data to a file
price_data = all_data.to_csv()
with open("price_data.csv", "w") as f:
  f.write(price_data)

#Download all available rating agencies recommendations
This process takes quite some time to complete. It can be skipped via using the next Colab file with analytics

In [14]:
#@title Analyst rating dataframe

#Recomendations will be fetched only for 
#80% of stocks with longest market presence
recom_data = pd.DataFrame()
assets = all_data.columns.values
assets = np.delete(assets,0)

def get_recom_data(recom_data, assets, start, end):
  """yfinance server limits output length to around 100 queries"""
  """so output has to be fetched in batches"""
  for index, ticker in enumerate(assets):
    if index >= start and index < end:
      yf_symbol = yf.Ticker(ticker)
      df_rec = yf_symbol.recommendations
      if df_rec is not None: 
        df_rec['Ticker'] = ticker
        df_rec = df_rec[['Ticker', "Action", "Firm",
                         "To Grade", "From Grade"]]
        recom_data = pd.concat([recom_data, df_rec])
  return recom_data

#Data is fetched in batches of 100 items
for i in range(5):
  recom_data = get_recom_data(recom_data, assets, i*100, (i+1)*100)
  print(f'{(i+1)*100} tickers fetched')

100 tickers fetched
200 tickers fetched
300 tickers fetched
400 tickers fetched
500 tickers fetched


In [15]:
#Save the data to the file
recom_csv = recom_data.to_csv()
with open("recom_csv.csv", "w") as f:
  f.write(recom_csv)

#Download all the stocks' number of shares outstanding 
This process takes quite some time to complete. It can be skipped via using the next Colab file with analytics

In [16]:
shares_now = {}

def fetch_shares(assets, shares_now):
  for ticker in assets:
    ticker_data = yf.Ticker(ticker)
    s_out = ticker_data.info
    if "sharesOutstanding" in s_out:
      shares_now[ticker] = s_out["sharesOutstanding"]
  return shares_now

for i in range(5):
  shares_now = fetch_shares(assets[i*100:(i+1)*100], shares_now)
  print(f"{(i+1)*100} tickers done")

print(shares_now)

100 tickers done
200 tickers done
300 tickers done
400 tickers done
500 tickers done
{'MMM': 576822016, 'ABT': 1772359936, 'ABBV': 1764829952, 'ABMD': 45189900, 'ACN': 635000000, 'ATVI': 772857024, 'ADBE': 479719008, 'AMD': 1202710016, 'AAP': 67854496, 'AES': 665131008, 'AFL': 702444992, 'A': 308310016, 'APD': 220894000, 'AKAM': 162704992, 'ALK': 123664000, 'ALB': 106457000, 'ARE': 134944000, 'ALXN': 218844992, 'ALGN': 78850400, 'ALLE': 92039000, 'LNT': 249644000, 'ALL': 304068000, 'GOOGL': 300644000, 'GOOG': 329867008, 'MO': 1858419968, 'AMZN': 500889984, 'AMCR': 1568480000, 'AEE': 247207008, 'AAL': 566489984, 'AEP': 496390016, 'AXP': 805201984, 'AIG': 861526016, 'AMT': 444212992, 'AWK': 181272000, 'AMP': 117978000, 'ABC': 204142000, 'AME': 230064992, 'AMGN': 582169024, 'APH': 299155008, 'ADI': 369560000, 'ANSS': 85884096}


In [17]:
#Save the data to a file
with open('shares_outstanding.pkl', 'wb') as f:
  pickle.dump(shares_now, f)